In [1]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 4 not upgraded.
Need to get 70.1 MB of archives.
After this operation, 257 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 75.0.3770.90-0ubuntu0.18.04.1 [1,112 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 75.0.3770

In [0]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import datetime as dt
import urllib.parse
import re
from tqdm import tqdm_notebook
import pandas as pd

In [0]:
options = webdriver.ChromeOptions()
options.add_argument ('--headless')
options.add_argument ('--no-sandbox')
options.add_argument ('--disable-dev-shm-usage')

browser = webdriver.Chrome ( 'chromedriver' , options = options)

In [0]:
# 검색을 위한 변수 설정 
startdate=dt.date(year=2018,month=1,day=1)
untildate=dt.date(year=2018,month=1,day=2)
enddate = dt.date(year=2018,month=12,day=31)
keyword = urllib.parse.quote_plus("랄라블라")

In [0]:
# tweet_bag=[]
tweet = []
daily_tweet = {}

In [6]:
while not enddate==startdate:
  
  print(startdate)
  daily_tweet[str(startdate)] = []

  url='https://twitter.com/search?l=ko&q='+keyword+'%20since%3A'+str(startdate)+'%20until%3A'+str(untildate)+'&src=typd'

  browser.get (url)

  html = browser.page_source
  soup=BeautifulSoup(html,'html.parser')

  lastHeight = browser.execute_script("return document.body.scrollHeight")

  while True:
    tweets=soup.find_all("p", {"class": "TweetTextSize"})

    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

    newHeight = browser.execute_script("return document.body.scrollHeight")
    
    if newHeight != lastHeight:
#       tweet_bag.append(tweets)
      daily_tweet[str(startdate)] += tweets
    else:
      startdate=untildate
      untildate+=dt.timedelta(days=1)
      break
  
    lastHeight = newHeight

2018-01-01
2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-06
2018-01-07
2018-01-08
2018-01-09
2018-01-10
2018-01-11
2018-01-12
2018-01-13
2018-01-14
2018-01-15
2018-01-16
2018-01-17
2018-01-18
2018-01-19
2018-01-20
2018-01-21
2018-01-22
2018-01-23
2018-01-24
2018-01-25
2018-01-26
2018-01-27
2018-01-28
2018-01-29
2018-01-30
2018-01-31
2018-02-01
2018-02-02
2018-02-03
2018-02-04
2018-02-05
2018-02-06
2018-02-07
2018-02-08
2018-02-09
2018-02-10
2018-02-11
2018-02-12
2018-02-13
2018-02-14
2018-02-15
2018-02-16
2018-02-17
2018-02-18
2018-02-19
2018-02-20
2018-02-21
2018-02-22
2018-02-23
2018-02-24
2018-02-25
2018-02-26
2018-02-27
2018-02-28
2018-03-01
2018-03-02
2018-03-03
2018-03-04
2018-03-05
2018-03-06
2018-03-07
2018-03-08
2018-03-09
2018-03-10
2018-03-11
2018-03-12
2018-03-13
2018-03-14
2018-03-15
2018-03-16
2018-03-17
2018-03-18
2018-03-19
2018-03-20
2018-03-21
2018-03-22
2018-03-23
2018-03-24
2018-03-25
2018-03-26
2018-03-27
2018-03-28
2018-03-29
2018-03-30
2018-03-31
2018-04-01

In [7]:
len(daily_tweet.keys())

364

In [10]:
daily_tweet['2018-01-02']

[<p class="TweetTextSize js-tweet-text tweet-text" data-aria-label-part="0" lang="ko"><strong>랄라블라</strong>........
 간판은 이쁜데 억케 불러야함?ㅋㅋ <a class="twitter-timeline-link u-hidden" data-pre-embedded="true" dir="ltr" href="https://t.co/pZKHQcEe49">pic.twitter.com/pZKHQcEe49</a></p>,
 <p class="TweetTextSize js-tweet-text tweet-text" data-aria-label-part="0" lang="ko"><strong>랄라블라</strong> 디자인 이쁜데 모르는 사람들 보면 라라라로 볼듯ㅋㅋㅋㅋㅋㅋㅋㅋㅋ <a class="twitter-timeline-link u-hidden" data-pre-embedded="true" dir="ltr" href="https://t.co/1vbazUhCsJ">pic.twitter.com/1vbazUhCsJ</a></p>,
 <p class="TweetTextSize js-tweet-text tweet-text" data-aria-label-part="0" lang="ko"><strong>랄라블라</strong> raravla가 아니라니 실몽입니다</p>,
 <p class="TweetTextSize js-tweet-text tweet-text" data-aria-label-part="0" lang="ko">이것도 적응되겠지
 
 GS 왓슨스 '<strong>랄라블라</strong>'로 브랜드 변경 추진 | 다음뉴스 <a class="twitter-timeline-link" data-expanded-url="http://v.media.daum.net/v/20171231180751544" dir="ltr" href="https://t.co/sHD7OPTa7p" rel="nofollow

In [0]:
def cleantweet(total_tweets_dict):
  keys = list(total_tweets_dict.keys())
  cleaned_tweet_dict = {}
  
  for key in tqdm_notebook(keys,desc = '일별할당'):
    cleaned_tweet = []
    cleaned_tweet_dict[key] = []
    tweets_value = total_tweets_dict[key]
    
    for tweet in tweets_value:
      tmp = re.sub('<.+?>', '', str(tweet), 0).strip()
      cleaned_tweet.append(tmp)
    cleaned_tweet_dict[key] += cleaned_tweet
    
  return cleaned_tweet_dict

In [13]:
cleaned_tweet_dict = cleantweet(daily_tweet)

In [14]:
cleaned_tweet_dict

{'2018-01-01': [],
 '2018-01-02': ['랄라블라........\n간판은 이쁜데 억케 불러야함?ㅋㅋ pic.twitter.com/pZKHQcEe49',
  '랄라블라 디자인 이쁜데 모르는 사람들 보면 라라라로 볼듯ㅋㅋㅋㅋㅋㅋㅋㅋㅋ pic.twitter.com/1vbazUhCsJ',
  '랄라블라 raravla가 아니라니 실몽입니다',
  "이것도 적응되겠지\n\nGS 왓슨스 '랄라블라'로 브랜드 변경 추진 | 다음뉴스 http://v.media.daum.net/v/20171231180751544\xa0…",
  '랄라랄라? 랄라블라?',
  '방금 랄라블라로 바뀐 왓슨스의 새 간판을 보고야 말았다',
  '헉 먕그님 랄라블라 알바생 되셨어..화이팅이에요....',
  '랄라블라ㅋㅋㅋㅋㅋ깜찍한 이름이에요...',
  '?? 왓슨스가 랄라블라로 바뀐다고?? 미쳤구나',
  '뭐야 왓슨스 진짜 랄라블라로 바뀜?',
  '랄라블라ㅋㅋㅋㅋㅅㅂㅋㅋㅋ',
  '헐 랄라블라 멘션이랑 착각했어',
  '천호님 랄라블라.....',
  '랄라블라',
  '친구들이랑 저기 가자고 할때도 이제 랄라블라 갈래?? 이래야겠어요 ㅋㅋㅋㅋㅋㅋ 웃음이 멈추질 않네요 진짜. . .',
  'ㄹㅋㄹㅋㄹㅋㄹㅋㄹㅋㅎ우리같이 랄라블라 갈래~?',
  '랄라블라~~마법주문같네여',
  '헐 진짜 랄라블라됨? 후.... 그럼 직원분들 앞으로 행사있을 때 안녕하세요 랄라블라 입니다!! 이럼?\n랄라부랄...',
  '아동복 진자 ㅋㅋㅋ 넘 웃겨요 방금 보니까 아직 플친 이름은 랄라블라로 안 바뀌었네요 그래두 좀 이따 바뀌겠죠? 휴우;;',
  '이름 너무 난잡하지 않나요 랄라블라, , , 만우절도 아니고 진심으로 했다는 게 너무 웃겨요 ㅋㅋㅋ'],
 '2018-01-03': ['왓슨스나 올리브영 롭스같은 이름들이 얼마나 무난하고 좋은 이름인지 알지못했어요 랄라블라를 만나기전까진... pic.twitter.com/R9MVrDDX7u',
  '봐봐 이제 왓슨스 랄라블라 됐서,, 벌써 

In [0]:
lalavla_tweet = pd.DataFrame.from_dict(cleaned_tweet_dict,orient='index')

In [0]:
lalavla_2018 = lalavla_tweet.transpose()

In [0]:
lalavla_2018.

In [29]:
lalavla_2018.to_csv('lalavla_2018.csv')

cp: target 'Competition/data' is not a directory


In [0]:
!cp lalavla_2018.csv ./gdrive/My\ Drive/GitHub/Weather\ Competition/

In [0]:
# import pickle

# with open('2018_lalavla.txt', 'wb') as f:
#      pickle.dump(cleaned_tweet_bag, f)

In [0]:
# with open('2018_lalavla.txt', 'rb') as f:
# ...     data = pickle.load(f) # 단 한줄씩 읽어옴